In [ ]:
def read_prep_write_AR6_fig9_26_netcdfs(infilename,veldomain=False):
    """
    Brings in the AR6 component NetCDF files and corrects the data arrangement to match that of the 
    AR5 and AR6 data that we are working with. Importantly, the data were transposed and also have
    a duplicate data column in the longitude dimension to force the wrap-around in matlab. 

    This function takes the name of the file that is to be processed and also a flag if the data are
    to be placed on the domain of the velocity data or the slightly more contrained domain for 
    mapping. Output is a xarray dataset with attributes taken from the original and additional
    attributes given the lat and lon coordinates.
    """
    #Rebuild this bag of arse from the ground up:
    # Create the xarray Dataset (code from MS copilot)
    dsin = xr.open_dataset(infilename)
    dsin = dsin.rename({'Longitude': 'lon', 'Latitude': 'lat',})
    ds = xr.Dataset(
        {
            "SL_change": (("lat", "lon"), np.transpose(dsin.SL_change[0:360,:].values)),
        },
        coords={
            "lon": (("lon",), dsin.lon[0:360].values),
            "lat": (("lat",), dsin.lat.values),
        },
    )
    ds = ds.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
    ds = ds.rio.write_crs("epsg:4326")
    #Add correct attributes for the lat/lon coordinates
    ds = add_latlon_attrs(ds)

    #Interpolate it to commonly used tenth degree grid.
    #ds = interp_canada_tenth_degree(ds,'nearest',veldomain)
    
    #Add the attributes from the parent data to the Data variable
    ds.SL_change.attrs['title'] = dsin.attrs['title']
    ds.SL_change.attrs['units'] = dsin.attrs['units']
    ds.SL_change.attrs['creator'] = dsin.attrs['creator']
    ds.SL_change.attrs['activity'] = dsin.attrs['activity']
    ds.SL_change.attrs['comments'] = dsin.attrs['comments']
    ds.SL_change.attrs['long_name'] = infilename.split('_')[-3:-1][1] + ' sea-level contribution'
    #Change the name of the dataset to prevent clobber when merged with other data
    #split the file on . and keep the second to last
    dsetname = infilename.split('_')[-3:-1][0]+'_'+infilename.split('_')[-3:-1][1] + '_slr'
    ds = ds.rename({'SL_change': dsetname})
    basename = infilename.split('.')[0]
    outfilename = basename + '_ll_fixed.nc'
    ds.to_netcdf(outfilename)
    return ds

def add_latlon_attrs(ds):
    """
    Simple code to add the correct attributes to an xarray spatial grid 
    with dimensions lat and lon.

    May write an extension of this function to correctly detect if lat/lon 
    exist in the file and modify the existing names to the conforming name lat/lon.
    The existince of a billion names for these ever-so-common grid dimensions
    is hyper annoying.

    Takes an xarray dataset as input and assigns coordinates.
    """
    commonlats = ['Lat','lat','latitude','Latitude','lats', 'Lats']
    commonlons = ['Lon','lon', 'Long', 'long', 'longitude','longitude','lons', 'Lons', 'longs', 'Longs']
    
    #Test to see if lat and lon exist in the names of the 
    if ('lat' in list(ds.coords)) & ('lon' in list(ds.coords)):
        #We have the required latitude and longitude names
        ds.lon.attrs['standard_name'] = 'longitude'
        ds.lon.attrs['long_name'] = 'longitude'
        ds.lon.attrs['units'] = 'degrees_east'
        ds.lon.attrs['axis'] = 'X'
        ds.lat.attrs['standard_name'] = 'latitude'
        ds.lat.attrs['long_name'] = 'latitude'
        ds.lat.attrs['units'] = 'degrees_north'
        ds.lat.attrs['axis'] = 'Y'
        return ds
    else:
        #For now, do nothing. Maybe change to alter the coord that is there to 
        #the desired name and then alter as above.
        return ds
    


In [ ]:
read_prep_write_AR6_fig9_26_netcdfs('C:/Users/fanslow/Work/SLR/Chapter-9/Plotting_code_and_data/Fig9_26_SL_regional/Plotted_Data/Fig9-26_data_ssp585_oceandynamics_map.nc')


def print_latnodedata(latnodes,quantile):
    print(ds.sel(locations=ds.locations.isin(latnodes),quantiles=quantile,years=2100).lat.values[0],
          ds.sel(locations=ds.locations.isin(latnodes),quantiles=quantile,years=2100).sea_level_change.values)
#    print(ds.sel(locations=ds.locations.isin(latnodes),quantiles=0.50,years=2100).sea_level_change.values)
quantile = 0.5    
latnodes = [173]
#latnodes = [1004102870,1004102880,1004102890,1004102900,1004102910,1004102920,1004102930]
print('----',ds.sel(locations=ds.locations.isin(latnodes),quantiles=quantile,years=2100).lon.values)   
print_latnodedata(latnodes,quantile)
latnodes = [i + 100000 for i in latnodes]
print_latnodedata(latnodes,quantile)
latnodes = [i + 100000 for i in latnodes]
print_latnodedata(latnodes,quantile)
latnodes = [i + 100000 for i in latnodes]
print_latnodedata(latnodes,quantile)
latnodes = [i + 100000 for i in latnodes]
print_latnodedata(latnodes,quantile)
# print(ds.sel(locations=ds.locations.isin([1004202880,1004202890,1004202900,1004202910,1004202920]),quantiles=0.50,years=2100).lat.values)
# print(ds.sel(locations=ds.locations.isin([1004202880,1004202890,1004202900,1004202910,1004202920]),quantiles=0.50,years=2100).lon.values)
# print(ds.sel(locations=ds.locations.isin([1004202880,1004202890,1004202900,1004202910,1004202920]),quantiles=0.50,years=2100).sea_level_change.values)

# print(ds.sel(locations=ds.locations.isin([1004302880,1004302890,1004302900,1004302910,1004302920]),quantiles=0.50,years=2100).lat.values)
# print(ds.sel(locations=ds.locations.isin([1004302880,1004302890,1004302900,1004302910,1004302920]),quantiles=0.50,years=2100).lon.values)
# print(ds.sel(locations=ds.locations.isin([1004302880,1004302890,1004302900,1004302910,1004302920]),quantiles=0.50,years=2100).sea_level_change.values)
